## playing with parallelism


In [1]:
print(system('hostname'))

[1] 0


In [7]:
print(system('hostname; uptime', intern=TRUE))


[1] "3ff288903d14"                                                          
[2] " 23:26:30 up 61 days,  6:49,  0 users,  load average: 0.06, 0.39, 0.34"


In [22]:
install.packages('doSNOW')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [25]:
library(snow)
library(doSNOW)

In [116]:
mat49 = matrix( NA, nrow=4, ncol=9)

for ( r in 1:4 ) {
    #print(c(a,a^2)) 
    for( c in 3:5) {
        mat49[r,c]=r*10 + c
    }
}

mat49

NA,NA,13,14,15,NA,NA,NA,NA
NA,NA,23,24,25,NA,NA,NA,NA
NA,NA,33,34,35,NA,NA,NA,NA
NA,NA,43,44,45,NA,NA,NA,NA


In [143]:
print("mat initialization")
mat = matrix( NA, nrow=4, ncol=9)
#mat
# there maybe some auto pivot of matrix from 4x1 to 1x4 as needed?
print("foreach... %do%")
foreach(r=1:4, .combine="rbind") %do% {
    foreach(c=3:5) %do% {
            mat[r,c] = r*10 + c
    }
}

print("mat result")
mat

[1] "mat initialization"
[1] "foreach... %do%"


result.1,13,14,15
result.2,23,24,25
result.3,33,34,35
result.4,43,44,45


[1] "mat result"


NA,NA,13,14,15,NA,NA,NA,NA
NA,NA,23,24,25,NA,NA,NA,NA
NA,NA,33,34,35,NA,NA,NA,NA
NA,NA,43,44,45,NA,NA,NA,NA


In [150]:
library(foreach)
library(doParallel)


# https://cran.r-project.org/web/packages/doParallel/vignettes/gettingstartedParallel.pdf
getDoParName()

#print("foreach... %dopar% ============= ")
mat = matrix( NA, nrow=4, ncol=9)



NumThread = detectCores() ## return number of cores on machine
print(c('detectCores', detectCores()))
#NumThread = 4
registerDoParallel(NumThread)


#clus1 = makeCluster( NumThread, outfile="clus1.out", type = "SOCK" )   ## create a cluster object

foreach(r=1:4, .combine="rbind") %do% {
        foreach(c=3:5) %do% {
            mat[r,c] = r*10 + c
            #mat[r,c] = r*10 + c
            #print(r)
        }
}

mat


[1] "doParallelMC"

[1] "detectCores" "16"         


[1] "doParallelMC"

result.1,13,14,15
result.2,23,24,25
result.3,33,34,35
result.4,43,44,45


NA,NA,13,14,15,NA,NA,NA,NA
NA,NA,23,24,25,NA,NA,NA,NA
NA,NA,33,34,35,NA,NA,NA,NA
NA,NA,43,44,45,NA,NA,NA,NA


In [154]:
library(doParallel)



#print("foreach... %dopar% ============= ")
mat = matrix( NA, nrow=4, ncol=9)


NumThread = detectCores() ## return number of cores on machine

#print(c('detectCores', detectCores()))

NumThread = 4



## see https://privefl.github.io/blog/a-guide-to-parallelism-in-r/
## under "Filling something in parallel"
clus1 = makeCluster( NumThread, outfile="clus1.out", type = "SOCK" )   ## create a cluster object

clusterCall(clus1, function() {
    library(doSNOW)
    registerDoSNOW(makeCluseter(8, type="SOCK"))
    ## code in here are dispatched to different node, memory is forked?  and later need to collect result?? 
    mat = matrix( NA, nrow=4, ncol=9)
    res1 = foreach(r=1:4, .combine="rbind") %do% {
        res2 = foreach(c=3:5) %do% {
            ## need to think of foreach like lapply.  it return a list, the side effects are done in parallelization!
            ## elementwise matrix update work while inside the foreach
            ## but mat is not "returned" to caller
            ## it would be better to think in terms of functional programming!
            mat[r,c] = r*10 + c
            #mat[r,c] = r*10 + c
            #print(r)
        }
    }
    #mat
    #result
    #print("res:")
    #res1
    #res2


})
            
print("mat:")
mat



# same foreach loop, compare result of $do% vs $dopar%, they should be identical

result.1,13,14,15
result.2,23,24,25
result.3,33,34,35
result.4,43,44,45
result.1,13,14,15
result.2,23,24,25
result.3,33,34,35
result.4,43,44,45
result.1,13,14,15
result.2,23,24,25
result.3,33,34,35


[1] "mat:"


NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,NA


In [120]:
mat49 = matrix( NA, nrow=4, ncol=9)

foreach ( r=1:4 )  %do% {
    #print(c(a,a^2)) 
    for( c in 3:5) {
        mat49[r,c]=r*10 + c
    }
}

mat49

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

NA,NA,13,14,15,NA,NA,NA,NA
NA,NA,23,24,25,NA,NA,NA,NA
NA,NA,33,34,35,NA,NA,NA,NA
NA,NA,43,44,45,NA,NA,NA,NA


In [37]:
library(doParallel)





NumThread = detectCores() ## return number of cores on machine
print(c('detectCores', detectCores()))

NumThread = 4


clus1 = makeCluster( NumThread, outfile="clus1.out", type = "SOCK" )   ## create a cluster object

clusterCall(clus1, function() {
    library(doSNOW)
    registerDoSNOW(makeCluseter(8, type="SOCK"))
    foreach(j=2:3) %dopar% {
        c(j, j^3)
    }
    
    
})

print("\n\n")

registerDoParallel(clus1)
foreach(i=1:4) %do% {
    c(i, i^2)
    foreach(j=2:3) %do% {
        c(j, j^3)
    }
    

}

    # the %dopar% says to run the foreach loop in parallel for each item
    # all console output inside cluster object will go to outfile 
    # there are eg combine=rbind to merge result

stopCluster(clus1)



[[1]]
[1] 1 1

[[2]]
[1] 2 8

[[3]]
[1]  3 27

[1] "detectCores" "16"         


[[1]]
[[1]][[1]]
[1] 2 8

[[1]][[2]]
[1]  3 27


[[2]]
[[2]][[1]]
[1] 2 8

[[2]][[2]]
[1]  3 27


[[3]]
[[3]][[1]]
[1] 2 8

[[3]][[2]]
[1]  3 27


[[4]]
[[4]][[1]]
[1] 2 8

[[4]][[2]]
[1]  3 27

[1] "\n\n"


[[1]]
[[1]][[1]]
[1] 2 8

[[1]][[2]]
[1]  3 27


[[2]]
[[2]][[1]]
[1] 2 8

[[2]][[2]]
[1]  3 27


[[3]]
[[3]][[1]]
[1] 2 8

[[3]][[2]]
[1]  3 27


[[4]]
[[4]][[1]]
[1] 2 8

[[4]][[2]]
[1]  3 27

In [79]:
c(1,2,3)
c(  c(1,2,3), c(10,20,30) )

[1] 1 2 3

[1]  1  2  3 10 20 30

In [141]:
# https://stackoverflow.com/questions/17508238/nested-foreach-loops-in-r-to-update-common-array
# foreach loops are used for their return value, like lapply. 
# In this way they are very different from for loops which are used for their side effects

## which is why the below don't print sequential result I expect!!
foreach(i=1:6) %do% {
    c(i, i^2)
    foreach(j=10:12) %do% {
        c(j, j^3)
    }
}

## .combine may help...  rbind, cbind, c,  ??? row-bind? col-bind,  c=??  the c() contat fn?
foreach(i=1:6, .combine='cbind') %:% {
    c(i, i^2)
    foreach(j=10:30, by=10, .combine='c') %do% {
        ##c(j, j^3, i+j)
        ##c(j, i, i+j)
        c(j, , j^3)
    }
}


# need to read:
# https://cran.r-project.org/web/packages/foreach/vignettes/nested.html

# foreach differs from a for loop in that its return is a list of values, whereas a for loop has no value and uses side effects to convey its result.



[[1]]
[[1]][[1]]
[1]   10 1000

[[1]][[2]]
[1]   11 1331

[[1]][[3]]
[1]   12 1728


[[2]]
[[2]][[1]]
[1]   10 1000

[[2]][[2]]
[1]   11 1331

[[2]][[3]]
[1]   12 1728


[[3]]
[[3]][[1]]
[1]   10 1000

[[3]][[2]]
[1]   11 1331

[[3]][[3]]
[1]   12 1728


[[4]]
[[4]][[1]]
[1]   10 1000

[[4]][[2]]
[1]   11 1331

[[4]][[3]]
[1]   12 1728


[[5]]
[[5]][[1]]
[1]   10 1000

[[5]][[2]]
[1]   11 1331

[[5]][[3]]
[1]   12 1728


[[6]]
[[6]][[1]]
[1]   10 1000

[[6]][[2]]
[1]   11 1331

[[6]][[3]]
[1]   12 1728

ERROR: Error in {: task 1 failed - "argument 2 is empty"


In [167]:

?mccollect

mcparallel {parallel},R Documentation
expr,"expression to evaluate (do not use any on-screen devices or GUI elements in this code, see mcfork for the inadvisability of using mcparallel with GUI front-ends and multi-threaded libraries). Raw vectors are reserved for internal use and cannot be returned, but the expression may evaluate e.g. to a list holding a raw vector. NULL should not be returned because it is used by mccollect to signal an error."
name,an optional name (character vector of length one) that can be associated with the job.
mc.set.seed,logical: see section ‘Random numbers’.
silent,if set to TRUE then all output on stdout will be suppressed (stderr is not affected).
mc.affinity,either a numeric vector specifying CPUs to restrict the child process to (1-based) or NULL to not modify the CPU affinity
mc.interactive,"logical, if TRUE or FALSE then the child process will be set as interactive or non-interactive respectively. If NA then the child process will inherit the interactive flag from the parent."
detached,"logical, if TRUE then the job is detached from the current session and cannot deliver any results back - it is used for the code side-effect only."
jobs,list of jobs (or a single job) to collect results for. Alternatively jobs can also be an integer vector of process IDs. If omitted collect will wait for all currently existing children.
wait,"if set to FALSE it checks for any results that are available within timeout seconds from now, otherwise it waits for all specified jobs to finish."
timeout,timeout (in seconds) to check for job results – applies only if wait is FALSE.
